# Importing libraries

In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV, StratifiedKFold, cross_val_score
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import SMOTENC
from imblearn.pipeline import Pipeline
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, space_eval
import shap

In [ ]:
!pip install optuna
import optuna

# Reading Data

In [ ]:
file_path = '/content/Sleep_health_and_lifestyle_dataset.csv'

if not os.path.exists(file_path):
    raise FileNotFoundError('파일을 찾을 수 없습니다.')

df = pd.read_csv(file_path)
print(df.shape)
print(df.columns.tolist())
print(df.head())
print(df.info())
print(df.describe())
print(df.isnull().sum())

# EDA

In [ ]:
print(df.select_dtypes(include=np.number).columns.tolist())

for col in df.select_dtypes(include=np.number).columns.tolist():
    plt.figure(figsize=(6,3))
    sns.histplot(df[col].dropna(), kde=True)
    plt.title(f'{col} Distribution')
    plt.show()

print(df.select_dtypes(exclude=np.number).columns.tolist())

for col in df.select_dtypes(exclude=np.number).columns.tolist():
    plt.figure(figsize=(5,3))
    df[col].value_counts(dropna=False).plot(kind='bar')
    plt.title(f'{col} Distribution')
    plt.show()

sns.heatmap(df.select_dtypes(include=np.number).corr(), annot=True, cmap='coolwarm')
plt.title("Heatmap")
plt.show()

# Data preprocessing

In [ ]:
target_col = 'Sleep Disorder'

if target_col not in df.columns:
    raise KeyError('Sleep Disorder가 없습니다.')

y_raw = df[target_col].fillna('Normal').astype(str)
le_target = LabelEncoder()
y = le_target.fit_transform(y_raw)

print('타겟 클래스:', le_target.classes_)
print('레이블 매핑:', dict(zip(le_target.classes_, le_target.transform(le_target.classes_))))

In [ ]:
X = df.copy()

if 'Person ID' not in X.columns:
   raise KeyError('Person ID가 없습니다.')

X.drop(columns=['Sleep Disorder', 'Person ID'], inplace=True)

bp = [c for c in X.columns if 'Blood Pressure' in c]

# Blood Pressure을 Systolic/Diastolic로 분리
if bp:
  bp_col = bp[0]
  bp = X[bp_col].astype(str).str.extract(r'(?P<Systolic>\d{2,3})\D+(?P<Diastolic>\d{2,3})')
  X['Systolic'] = pd.to_numeric(bp['Systolic'], errors='coerce')
  X['Diastolic'] = pd.to_numeric(bp['Diastolic'], errors='coerce')
  X.drop(columns=[bp_col], inplace=True)
else:
  X['Systolic'] = np.nan
  X['Diastolic'] = np.nan

# 혈압 상태 카테고리 분류 함수
def bp_category(sys, dia):
    if pd.isna(sys) or pd.isna(dia): return 'Unknown'
    if sys >= 140 or dia >= 90: return 'High'
    if (130 <= sys <= 139) or (80 <= dia <= 89): return 'Borderline'
    return 'Normal'

# 혈압 상태 카테고리 분류 적용
X['BP Category'] = X.apply(lambda r: bp_category(r['Systolic'], r['Diastolic']), axis=1)
X.drop(columns=['Systolic', 'Diastolic'], inplace=True)

In [ ]:
cat_cols = X.select_dtypes(exclude=np.number).columns.tolist()

# 레이블 인코딩을 적용
le_dict = {}

for col in cat_cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col].astype(str))
    le_dict[col] = le # 역변환을 위해 인코더 저장

In [ ]:
# 학습/테스트 데이터 분할, 클래스 비율 유지
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# 테스트 데이터에서 검증 데이터 추가 분할(과적합 방지용)
X_tr, X_val, y_tr, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42, stratify=y_train
)

print(f'학습 데이터: {X_tr.shape}, 검증 데이터: {X_val.shape}, 테스트 데이터: {X_test.shape}')

categorical_features = [X.columns.get_loc(c) for c in cat_cols]

# SMOTENC 설정
smote_nc = SMOTENC(
    categorical_features=categorical_features,
    random_state=42
)

# 학습 데이터를 대상으로 클래스 불균형 처리
# 연속형 + 범주형 데이터 혼합 환경에서 오버샘플링을 수행
X_tr_res, y_tr_res = smote_nc.fit_resample(X_tr, y_tr)

print(f'학습 데이터(SMOTE 적용 후): {X_tr_res.shape}')

# Data modeling

In [ ]:
results = {} # 모델별 하이퍼파라미터 실험 결과 저장

xgb_baseline = XGBClassifier(
    n_estimators=150,
    learning_rate=0.1,
    max_depth=5,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    eval_metric='mlogloss',
    objective='multi:softprob',
    num_class=3
)

xgb_baseline.fit(X_tr, y_tr, eval_set=[(X_tr, y_tr), (X_val, y_val)], verbose=False)
pred_xgb_baseline = xgb_baseline.predict(X_test)

results['XGB_Baseline'] = {
    'model': xgb_baseline,
    'pred': pred_xgb_baseline,
    'pred_proba': xgb_baseline.predict_proba(X_test),
    'accuracy': accuracy_score(y_test, pred_xgb_baseline),
    'f1_macro': f1_score(y_test, pred_xgb_baseline, average='macro'),
    'precision_macro': precision_score(y_test, pred_xgb_baseline, average='macro'),
    'recall_macro': recall_score(y_test, pred_xgb_baseline, average='macro')
}

In [ ]:
xgb_baseline_result = xgb_baseline.evals_result_
epochs = len(xgb_baseline_result['validation_0']['mlogloss'])

plt.title('Learning Curve')
plt.plot(range(epochs), xgb_baseline_result['validation_0']['mlogloss'], label='Train')
plt.plot(range(epochs), xgb_baseline_result['validation_1']['mlogloss'], label='Valid')
plt.xlabel('epochs')
plt.ylabel('mlogloss')
plt.legend()
plt.show()

In [ ]:
xgb_tuned = XGBClassifier(
    n_estimators=300,
    learning_rate=0.01,
    max_depth=3,
    min_child_weight=3,
    subsample=0.8,
    colsample_bytree=0.65,
    random_state=42,
    eval_metric='mlogloss',
    early_stopping_rounds=50,
    objective='multi:softprob',
    num_class=3
)

xgb_tuned.fit(X_tr, y_tr, eval_set=[(X_tr, y_tr), (X_val, y_val)], verbose=False)
pred_xgb_tuned = xgb_tuned.predict(X_test)

results['XGB_Tuned'] = {
    'model': xgb_tuned,
    'pred': pred_xgb_tuned,
    'pred_proba': xgb_tuned.predict_proba(X_test),
    'accuracy': accuracy_score(y_test, pred_xgb_tuned),
    'f1_macro': f1_score(y_test, pred_xgb_tuned, average='macro'),
    'precision_macro': precision_score(y_test, pred_xgb_tuned, average='macro'),
    'recall_macro': recall_score(y_test, pred_xgb_tuned, average='macro')
}

In [ ]:
xgb_tuned_result = xgb_tuned.evals_result_
epochs = len(xgb_tuned_result['validation_0']['mlogloss'])

plt.title('Learning Curve')
plt.plot(range(epochs), xgb_tuned_result['validation_0']['mlogloss'], label='Train')
plt.plot(range(epochs), xgb_tuned_result['validation_1']['mlogloss'], label='Valid')
plt.xlabel('epochs')
plt.ylabel('mlogloss')
plt.legend()
plt.show()

In [ ]:
# RandomizedSearchCV를 위한 하이퍼파라미터 탐색 범위 설정
param_dist = {
    'n_estimators': [300, 500, 700],
    'learning_rate': [0.01, 0.03, 0.05, 0.1],
    'max_depth': [2, 3, 4, 5],
    'min_child_weight': [2, 3, 5, 7],
    'subsample': [0.7, 0.8, 0.9],
    'colsample_bytree': [0.7, 0.8, 0.9],
}

xgb_random_search = RandomizedSearchCV(
    XGBClassifier(
        random_state=42,
        eval_metric='mlogloss',
        objective='multi:softprob',
        num_class=3
    ),
    param_distributions=param_dist,
    n_iter=30,
    scoring='f1_macro',
    cv=5,
    random_state=42,
)

xgb_random_search.fit(X_train, y_train)
pred_xgb_random_search = xgb_random_search.predict(X_test)

results['XGB_RandomSearch'] = {
    'model': xgb_random_search,
    'pred': pred_xgb_random_search,
    'pred_proba': xgb_random_search.predict_proba(X_test),
    'accuracy': accuracy_score(y_test, pred_xgb_random_search),
    'f1_macro': f1_score(y_test, pred_xgb_random_search, average='macro'),
    'precision_macro': precision_score(y_test, pred_xgb_random_search, average='macro'),
    'recall_macro': recall_score(y_test, pred_xgb_random_search, average='macro')
}

print(f'최적 파라미터: {xgb_random_search.best_params_}')

In [ ]:
# GridSearchCV를 위한 하이퍼파라미터 탐색 범위 설정
param_grid = {
    'n_estimators': [300, 500],
    'learning_rate': [0.01, 0.05],
    'max_depth': [2, 3, 4],
    'min_child_weight': [3, 5],
    'subsample': [0.7, 0.8],
    'colsample_bytree': [0.7, 0.8],
}

xgb_grid_search = GridSearchCV(
    XGBClassifier(
        random_state=42,
        eval_metric='mlogloss',
        objective='multi:softprob',
        num_class=3
    ),
    param_grid=param_grid,
    scoring='f1_macro',
    cv=5,
)

xgb_grid_search.fit(X_train, y_train)
pred_xgb_grid_search = xgb_grid_search.predict(X_test)

results['XGB_GridSearch'] = {
    'model': xgb_grid_search,
    'pred': pred_xgb_grid_search,
    'pred_proba': xgb_grid_search.predict_proba(X_test),
    'accuracy': accuracy_score(y_test, pred_xgb_grid_search),
    'f1_macro': f1_score(y_test, pred_xgb_grid_search, average='macro'),
    'precision_macro': precision_score(y_test, pred_xgb_grid_search, average='macro'),
    'recall_macro': recall_score(y_test, pred_xgb_grid_search, average='macro')
}


print(f'최적 파라미터: {xgb_grid_search.best_params_}')


In [ ]:
# Hyperopt를 위한 하이퍼파라미터 탐색 범위 설정
# 베이지안 최적화 기반 탐색
hyperopt_space = {
    'n_estimators': hp.quniform('n_estimators', 250, 500, 50),
    'learning_rate': hp.uniform('learning_rate', 0.03, 0.15),
    'max_depth': hp.quniform('max_depth', 2, 4, 1),
    'min_child_weight': hp.quniform('min_child_weight', 4, 8, 1),
    'subsample': hp.uniform('subsample', 0.6, 0.8),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.6, 0.75),
}

# Hyperopt를 위한 하이퍼파라미터 탐색 목적 함수 정의
def objective_hyperopt(params):
    params['n_estimators'] = int(params['n_estimators'])
    params['max_depth'] = int(params['max_depth'])
    params['min_child_weight'] = int(params['min_child_weight'])
    model = XGBClassifier(
        **params,
        random_state=42,
        eval_metric='mlogloss',
        objective='multi:softprob',
        num_class=3
    )
    score = cross_val_score(model, X_train, y_train, cv=5, scoring='f1_macro').mean()
    return {'loss': -score, 'status': STATUS_OK}

trials = Trials()
best_params_hyperopt = fmin(
    fn=objective_hyperopt,
    space=hyperopt_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=trials,
    rstate=np.random.default_rng(42),
)

best_params_hyperopt['n_estimators'] = int(best_params_hyperopt['n_estimators'])
best_params_hyperopt['max_depth'] = int(best_params_hyperopt['max_depth'])
best_params_hyperopt['min_child_weight'] = int(best_params_hyperopt['min_child_weight'])

xgb_hyperopt = XGBClassifier(
    **best_params_hyperopt,
    random_state=42,
    eval_metric='mlogloss',
    objective='multi:softprob',
    num_class=3,
    early_stopping_rounds=30,
)

xgb_hyperopt.fit(X_tr, y_tr, eval_set=[(X_val, y_val)], verbose=False)
pred_xgb_hyperopt = xgb_hyperopt.predict(X_test)

results['XGB_Hyperopt'] = {
    'model': xgb_hyperopt,
    'pred': pred_xgb_hyperopt,
    'pred_proba': xgb_hyperopt.predict_proba(X_test),
    'accuracy': accuracy_score(y_test, pred_xgb_hyperopt),
    'f1_macro': f1_score(y_test, pred_xgb_hyperopt, average='macro'),
    'precision_macro': precision_score(y_test, pred_xgb_hyperopt, average='macro'),
    'recall_macro': recall_score(y_test, pred_xgb_hyperopt, average='macro')
}

best_params = space_eval(hyperopt_space, best_params_hyperopt)
print(f'최적 파라미터 {best_params}')


In [ ]:
optuna.logging.set_verbosity(optuna.logging.WARNING)

# Optuna를 위한 하이퍼파라미터 탐색 목적 함수 정의
# 베이지안 최적화 기반 탐색
def objective_optuna(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 250, 500, step=50),
        'learning_rate': trial.suggest_float('learning_rate', 0.03, 0.15),
        'max_depth': trial.suggest_int('max_depth', 2, 4),
        'min_child_weight': trial.suggest_int('min_child_weight', 4, 8),
        'subsample': trial.suggest_float('subsample', 0.6, 0.8),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 0.75),
        'random_state': 42,
        'eval_metric': 'mlogloss',
        'objective':'multi:softprob',
        'num_class': 3
    }

    xgb_optuna = XGBClassifier(**params)
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(xgb_optuna, X_train, y_train, cv=cv, scoring='f1_macro').mean()

    return scores

study = optuna.create_study(direction='maximize')
study.optimize(objective_optuna, n_trials=50)

xgb_optuna = XGBClassifier(
    **study.best_params,
    random_state=42,
    eval_metric='mlogloss',
    early_stopping_rounds=30,
    objective='multi:softprob',
    num_class=3,
)

xgb_optuna.fit(X_tr, y_tr, eval_set=[(X_val, y_val)], verbose=False)
pred_xgb_optuna = xgb_optuna.predict(X_test)

results['XGB_Optuna'] = {
    'model': xgb_optuna,
    'pred': pred_xgb_optuna,
    'pred_proba': xgb_optuna.predict_proba(X_test),
    'accuracy': accuracy_score(y_test, pred_xgb_optuna),
    'f1_macro': f1_score(y_test, pred_xgb_optuna, average='macro'),
    'precision_macro': precision_score(y_test, pred_xgb_optuna, average='macro'),
    'recall_macro': recall_score(y_test, pred_xgb_optuna, average='macro')
}

print(f'최적 파라미터: {study.best_params}')

In [ ]:
xgb_baseline_smote = Pipeline([
    ('smote', smote_nc),
    ('model', XGBClassifier(
        n_estimators=150,
        learning_rate=0.1,
        max_depth=5,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        eval_metric='mlogloss',
        objective='multi:softprob',
        num_class=3
    ))
])

xgb_baseline_smote.fit(X_tr, y_tr)
pred_xgb_baseline = xgb_baseline_smote.predict(X_test)


results['XGB_Baseline_SMOTE'] = {
    'model': xgb_baseline_smote,
    'pred': pred_xgb_baseline,
    'pred_proba': xgb_baseline_smote.predict_proba(X_test),
    'accuracy': accuracy_score(y_test, pred_xgb_baseline),
    'f1_macro': f1_score(y_test, pred_xgb_baseline, average='macro'),
    'precision_macro': precision_score(y_test, pred_xgb_baseline, average='macro'),
    'recall_macro': recall_score(y_test, pred_xgb_baseline, average='macro')
}

In [ ]:
xgb_tuned_smote = Pipeline([
    ('smote', smote_nc),
    ('model', XGBClassifier(
        n_estimators=500,
        learning_rate=0.08,
        max_depth=3,
        min_child_weight=7,
        subsample=0.7,
        colsample_bytree=0.65,
        random_state=42,
        eval_metric='mlogloss',
        objective='multi:softprob',
        num_class=3
    ))
])

xgb_tuned_smote.fit(X_tr, y_tr)
pred_xgb_tuned_smote = xgb_tuned_smote.predict(X_test)

results['XGB_Tuned_SMOTE'] = {
    'model': xgb_tuned_smote,
    'pred': pred_xgb_tuned_smote,
    'pred_proba': xgb_tuned_smote.predict_proba(X_test),
    'accuracy': accuracy_score(y_test, pred_xgb_tuned_smote),
    'f1_macro': f1_score(y_test, pred_xgb_tuned_smote, average='macro'),
    'precision_macro': precision_score(y_test, pred_xgb_tuned_smote, average='macro'),
    'recall_macro': recall_score(y_test, pred_xgb_tuned_smote, average='macro')
}

In [ ]:
param_dist = {
    'model__n_estimators': [300, 500, 700],
    'model__learning_rate': [0.01, 0.03, 0.05, 0.1],
    'model__max_depth': [2, 3, 4, 5],
    'model__min_child_weight': [2, 3, 5, 7],
    'model__subsample': [0.7, 0.8, 0.9],
    'model__colsample_bytree': [0.7, 0.8, 0.9],
}

xgb_random_search_smote = RandomizedSearchCV(
    Pipeline([
        ('smote', smote_nc),
        ('model', XGBClassifier(
            eval_metric='mlogloss',
            objective='multi:softprob',
            num_class=3,
            random_state=42
        ))
    ]),
    param_distributions=param_dist,
    n_iter=50,
    scoring='f1_macro',
    cv=5,
    random_state=42
)

xgb_random_search_smote.fit(X_train, y_train)
pred_xgb_random_smote = xgb_random_search_smote.predict(X_test)

results['XGB_RandomSearch_SMOTE'] = {
    'model': xgb_random_search_smote,
    'pred': pred_xgb_random_smote,
    'pred_proba': xgb_random_search_smote.predict_proba(X_test),
    'accuracy': accuracy_score(y_test, pred_xgb_random_smote),
    'f1_macro': f1_score(y_test, pred_xgb_random_smote, average='macro'),
    'precision_macro': precision_score(y_test, pred_xgb_random_smote, average='macro'),
    'recall_macro': recall_score(y_test, pred_xgb_random_smote, average='macro')
}

print(f'최적 파라미터: {xgb_random_search_smote.best_params_}')

In [ ]:
param_grid = {
    'model__n_estimators': [300, 500],
    'model__learning_rate': [0.01, 0.05],
    'model__max_depth': [2, 3, 4],
    'model__min_child_weight': [3, 5],
    'model__subsample': [0.7, 0.8],
    'model__colsample_bytree': [0.7, 0.8]
}


xgb_grid_smote = GridSearchCV(
    Pipeline([
        ('smote', smote_nc),
        ('model', XGBClassifier(
            eval_metric='mlogloss',
            objective='multi:softprob',
            num_class=3,
            random_state=42
        ))
    ]),
    param_grid=param_grid,
    scoring='f1_macro',
    cv=5
)

xgb_grid_smote.fit(X_train, y_train)
pred_xgb_grid_smote = xgb_grid_smote.predict(X_test)

results['XGB_GridSearch_SMOTE'] = {
    'model': xgb_grid_smote,
    'pred': pred_xgb_grid_smote,
    'pred_proba': xgb_grid_smote.predict_proba(X_test),
    'accuracy': accuracy_score(y_test, pred_xgb_grid_smote),
    'f1_macro': f1_score(y_test, pred_xgb_grid_smote, average='macro'),
    'precision_macro': precision_score(y_test, pred_xgb_grid_smote, average='macro'),
    'recall_macro': recall_score(y_test, pred_xgb_grid_smote, average='macro')
}


print(f'최적 파라미터: {xgb_grid_smote.best_params_}')


# Models evaluation

In [ ]:
results_df = pd.DataFrame({
    'Model': list(results.keys()),
    'Accuracy': [v['accuracy'] for v in results.values()],
    'F1_Macro': [v['f1_macro'] for v in results.values()],
    'Recall_Macro': [v['recall_macro'] for v in results.values()],
    'Precision_Macro': [v['precision_macro'] for v in results.values()],
})

w_f1 = 0.5 # 불균형 데이터 대응을 위해 가중치를 가장 높게 설정
w_recall = 0.4
w_precision = 0.1

results_df['Weight_Score'] = (
    results_df['F1_Macro'] * w_f1 +
    results_df['Recall_Macro'] * w_recall +
    results_df['Precision_Macro'] * w_precision
)

results_df = results_df.sort_values('Weight_Score', ascending=False)
print(results_df)

for metric in ['Weight_Score', 'F1_Macro', 'Recall_Macro', 'Precision_Macro', 'Accuracy']:
    plt.figure(figsize=(8, 5))
    sns.barplot(x=metric, y='Model', data=results_df)
    plt.title(f'Model Comparison: {metric}')
    plt.tight_layout()
    plt.show()

In [ ]:
best_model_name = results_df.iloc[0]['Model']
best_model = results[best_model_name]['model']
best_pred = results[best_model_name]['pred']

print(f'Best Model: {best_model_name}')

# RandomizedSearchCV이면 best_estimator_ 사용
if hasattr(best_model, 'best_estimator_'):
    final_model = best_model.best_estimator_
else:
    final_model = best_model

print('\nClassification report:')
print(classification_report(y_test, best_pred, target_names=le_target.classes_))

cm = confusion_matrix(y_test, best_pred)
plt.figure(figsize=(6,4))
sns.heatmap(cm, annot=True, fmt='d', xticklabels=le_target.classes_, yticklabels=le_target.classes_, cmap='Blues')
plt.xlabel('Predicted');
plt.ylabel('True');
plt.title('Confusion Matrix')
plt.show()

importances = pd.Series(final_model.feature_importances_, index=X.columns).sort_values(ascending=False)
print('Feature Importances:\n', importances)

plt.figure(figsize=(8,5))
importances.plot(kind='bar')
plt.title('Feature Importances (All Features)')
plt.tight_layout()
plt.show()